In [65]:
from sklearn.datasets import load_breast_cancer
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## Fetch wine dataset

In [66]:
breast_cancer = load_breast_cancer()
X, y = breast_cancer.data, breast_cancer.target

X = torch.as_tensor(X, dtype = torch.float)
y = torch.as_tensor(y, dtype = torch.float)

## Split

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size= 0.5)


## Create Model

In [71]:
class MyModel(nn.Module):
    def __init__(self, hidden = [64, 32, 16]):
        super().__init__()

        self.input = nn.Linear(X.shape[1], hidden[0])
        self.hiddens = []
        for i in range(len(hidden)-1):
            self.hiddens.append(nn.Linear(hidden[i], hidden[i+1]))
        
        self.dropout = nn.Dropout(0.5)
        self.output = nn.Linear(hidden[-1], 1)
    
    def forward(self, x):
        out = nn.ReLU()(self.input(x))
        for layer in self.hiddens:
            out = nn.ReLU()(layer(out))

        out = self.dropout(out)

        return nn.Sigmoid()(self.output(out))
    

model = MyModel([512, 256, 128, 64, 32])
print(model, model.hiddens)

# Test architecture
model(X_train[0])

# Test shapes
print(model(X_train[0]).shape, y_train[0].shape)

MyModel(
  (input): Linear(in_features=30, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (output): Linear(in_features=32, out_features=1, bias=True)
) [Linear(in_features=512, out_features=256, bias=True), Linear(in_features=256, out_features=128, bias=True), Linear(in_features=128, out_features=64, bias=True), Linear(in_features=64, out_features=32, bias=True)]
torch.Size([1]) torch.Size([])


## Criterion and Optimizer

In [72]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2, momentum = 0.9)

## Train

In [73]:
EPOCHS = 100

for epoch in range(1, EPOCHS + 1):
    # Set model to train state
    model.train()

    # Predict
    y_pred = model(X_train).squeeze()

    # Compute loss
    loss = criterion(y_pred, y_train)

    # Reset gradients
    model.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimizer.step()

    # Compute accuracy
    train_acc = (y_pred.round() == y_train).sum() / len(X_train)

    # Validation accuracy
    y_val_pred = model(X_val).squeeze()
    val_acc = (y_val_pred.round() == y_val).sum() / len(X_val)

    print(f"Epoch {epoch}, loss: {(loss.item() / len(X_train)):.4f}, accuracy = {train_acc:.4f}, val_acc = {val_acc:.4f}")


Epoch 1, loss: 0.0031, accuracy = 0.5455, val_acc = 0.5175
Epoch 2, loss: 0.0024, accuracy = 0.5513, val_acc = 0.5000
Epoch 3, loss: 0.0023, accuracy = 0.4839, val_acc = 0.4298
Epoch 4, loss: 0.0025, accuracy = 0.4076, val_acc = 0.4561
Epoch 5, loss: 0.0023, accuracy = 0.4194, val_acc = 0.5088
Epoch 6, loss: 0.0022, accuracy = 0.4018, val_acc = 0.5263
Epoch 7, loss: 0.0021, accuracy = 0.4311, val_acc = 0.6228
Epoch 8, loss: 0.0020, accuracy = 0.6070, val_acc = 0.6053
Epoch 9, loss: 0.0021, accuracy = 0.6100, val_acc = 0.4825
Epoch 10, loss: 0.0020, accuracy = 0.6540, val_acc = 0.5965
Epoch 11, loss: 0.0019, accuracy = 0.6481, val_acc = 0.6579
Epoch 12, loss: 0.0018, accuracy = 0.6452, val_acc = 0.5877
Epoch 13, loss: 0.0019, accuracy = 0.5513, val_acc = 0.6579
Epoch 14, loss: 0.0019, accuracy = 0.5484, val_acc = 0.6930
Epoch 15, loss: 0.0018, accuracy = 0.6158, val_acc = 0.6316
Epoch 16, loss: 0.0017, accuracy = 0.7097, val_acc = 0.7719
Epoch 17, loss: 0.0017, accuracy = 0.7449, val_ac